# Trending detector - Web Scrapper

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

## multi page web scrapping

### Coletando atributos

In [2]:
import requests
from bs4 import BeautifulSoup as bs

first = 1
last_page = 150

URL = 'https://mercadoeconsumo.com.br/category/noticias/page/'

In [3]:
%%time

df = pd.DataFrame()
noticias = []

for page in tqdm(range(first, last_page)):
    req = requests.get(URL + str(page) + '/')
    soup = bs(req.text, 'html.parser')

    # Percorrendo todos os elementos com a classe "jeg_post_title"
    for titulo in soup.find_all("h3", class_="jeg_post_title"):

        # coletando o texto do título
        titulo_texto = titulo.get_text()
        titulo_texto = re.sub(r'\n', '', titulo_texto)

        # coletando o link do título
        link = titulo.find("a")["href"]

        # adicionando as informações na lista
        noticias.append({"titulo": titulo_texto, "link": link})

        df = pd.DataFrame(noticias)
        df = df.drop_duplicates('titulo').reset_index(drop=True)

# import math

# batch_size = 100 # Define o tamanho do batch
# batches = range(math.ceil(last_page/batch_size)) # Cria uma lista com o número de batches

# df = pd.DataFrame()
# noticias = []

# for batch in tqdm(batches):
#     batch_start = batch * batch_size + first
#     batch_end = min((batch + 1) * batch_size + first, last_page)

#     for page in range(batch_start, batch_end):
#         req = requests.get(URL + str(page) + '/')
#         soup = bs(req.text, 'html.parser')

#         # Percorrendo todos os elementos com a classe "jeg_post_title"
#         for titulo in soup.find_all("h3", class_="jeg_post_title"):
        
#             # coletando o texto do título
#             titulo_texto = titulo.get_text()
#             titulo_texto = re.sub(r'\n', '', titulo_texto)

#             # coletando o link do título
#             link = titulo.find("a")["href"]

#             # adicionando as informações na lista
#             noticias.append({"titulo": titulo_texto, "link": link})

#     df = pd.DataFrame(noticias)
#     df = df.drop_duplicates('titulo').reset_index(drop=True)
#     noticias = [] # Limpa a lista de notícias após cada iteração de batch


100%|█████████████████████████████████████████| 149/149 [16:07<00:00,  6.49s/it]

CPU times: user 18.9 s, sys: 301 ms, total: 19.3 s
Wall time: 16min 7s


In [4]:
df

,titulo,link
0,Mercado financeiro eleva projeção da inflação ...,https://mercadoeconsumo.com.br/06/02/2023/econ...
1,Mercado Livre prepara estreia no BBB e mira no...,https://mercadoeconsumo.com.br/06/02/2023/noti...
2,Lula avalia elevar isenção do IR a dois mínimo...,https://mercadoeconsumo.com.br/06/02/2023/econ...
3,PepsiCo segue rumo autonomia hídrica com 100 d...,https://mercadoeconsumo.com.br/06/02/2023/inov...
4,Fintechs e Energytechs são as startups que mai...,https://mercadoeconsumo.com.br/06/02/2023/noti...
...,...,...
1487,Bassar Pet Food tem desafio à frente após mort...,https://mercadoeconsumo.com.br/18/09/2022/econ...
1488,Remédios do Farmácia Popular cortados por Bols...,https://mercadoeconsumo.com.br/17/09/2022/econ...
1489,Natura lidera baixas da B3 após confirmar uniã...,https://mercadoeconsumo.com.br/17/09/2022/econ...
1490,Consumidor pós-pandemia espera experiências al...,https://mercadoeconsumo.com.br/17/09/2022/econ...


In [ ]:
df_pt1

### Coletando notícia de cada link

In [5]:
%%time
# conteudo_pag = pd.DataFrame()

# for link in tqdm(df['link']):
#     URL = link
#     req = requests.get(URL)
#     soup = bs(req.text, 'html.parser')

#     # coletando data do link
#     data = soup.find('div', class_= 'jeg_meta_date')
#     data = data.get_text()
#     data = re.sub(r'\n', '', data)
    
#     #lista_data.append({'data': data})

#     # coletando conteudo do link
#     conteudo = soup.find('div', class_='content-inner')
#     conteudo = conteudo.get_text()
#     conteudo = re.sub(r'\n', '', conteudo)
#     conteudo = re.sub(r'Compartilhe isso:FacebookLinkedInTwitterWhatsAppTelegramPinterestMaisImprimirCurtir isso:Curtir Carregando...', '', conteudo)

#     #conteudo_pag.append({"data": data, "conteudo": conteudo})
#     conteudo_pag = conteudo_pag.append({'data': data, 'conteudo': conteudo}, ignore_index=True)

import math

batch_size = 200 # Define o tamanho do batch
batches = range(math.ceil(len(df['link'])/batch_size)) # Cria uma lista com o número de batches
# batches

conteudo_pag = pd.DataFrame()

for batch in tqdm(batches):
    batch_start = batch * batch_size
    batch_end = min((batch + 1) * batch_size, len(df['link']))

    for link in df['link'][batch_start:batch_end]:
        URL = link
        req = requests.get(URL)
        soup = bs(req.text, 'html.parser')

        # coletando data do link
        data = soup.find('div', class_= 'jeg_meta_date')
        data = data.get_text()
        data = re.sub(r'\n', '', data)

        # coletando conteudo do link
        conteudo = soup.find('div', class_='content-inner')
        conteudo = conteudo.get_text()
        conteudo = re.sub(r'\n', '', conteudo)
        conteudo = re.sub(r'Compartilhe isso:FacebookLinkedInTwitterWhatsAppTelegramPinterestMaisImprimirCurtir isso:Curtir Carregando...', '', conteudo)

        conteudo_pag = conteudo_pag.append({'data': data, 'conteudo': conteudo}, ignore_index=True)

100%|█████████████████████████████████████████| 8/8 [2:41:43<00:00, 1212.89s/it]

CPU times: user 2min 34s, sys: 1.74 s, total: 2min 35s
Wall time: 2h 41min 43s


In [6]:
conteudo_pag

,data,conteudo
0,6 de fevereiro de 2023,A previsão do mercado financeiro para o Índice...
1,6 de fevereiro de 2023,Prestes a estrear como patrocinador do Big Bro...
2,6 de fevereiro de 2023,Depois de prometer na campanha isentar do Impo...
3,6 de fevereiro de 2023,"A PepsiCo, uma das maiores empresas de aliment..."
4,6 de fevereiro de 2023,De acordo com uma pesquisa realizada pela EqSe...
...,...,...
1487,18 de setembro de 2022,"Da noite para o dia, a Bassar Pet Food se torn..."
1488,17 de setembro de 2022,Os remédios oferecidos de graça pelo programa ...
1489,17 de setembro de 2022,"A Natura & Co. sofreu, ontem, uma forte queda,..."
1490,17 de setembro de 2022,Após as transformações causadas pela pandemia ...


## Agregando DFs

In [7]:
df = pd.concat([df, conteudo_pag], axis=1)

In [9]:
df

,titulo,link,data,conteudo
0,Mercado financeiro eleva projeção da inflação ...,https://mercadoeconsumo.com.br/06/02/2023/econ...,6 de fevereiro de 2023,A previsão do mercado financeiro para o Índice...
1,Mercado Livre prepara estreia no BBB e mira no...,https://mercadoeconsumo.com.br/06/02/2023/noti...,6 de fevereiro de 2023,Prestes a estrear como patrocinador do Big Bro...
2,Lula avalia elevar isenção do IR a dois mínimo...,https://mercadoeconsumo.com.br/06/02/2023/econ...,6 de fevereiro de 2023,Depois de prometer na campanha isentar do Impo...
3,PepsiCo segue rumo autonomia hídrica com 100 d...,https://mercadoeconsumo.com.br/06/02/2023/inov...,6 de fevereiro de 2023,"A PepsiCo, uma das maiores empresas de aliment..."
4,Fintechs e Energytechs são as startups que mai...,https://mercadoeconsumo.com.br/06/02/2023/noti...,6 de fevereiro de 2023,De acordo com uma pesquisa realizada pela EqSe...
...,...,...,...,...
1487,Bassar Pet Food tem desafio à frente após mort...,https://mercadoeconsumo.com.br/18/09/2022/econ...,18 de setembro de 2022,"Da noite para o dia, a Bassar Pet Food se torn..."
1488,Remédios do Farmácia Popular cortados por Bols...,https://mercadoeconsumo.com.br/17/09/2022/econ...,17 de setembro de 2022,Os remédios oferecidos de graça pelo programa ...
1489,Natura lidera baixas da B3 após confirmar uniã...,https://mercadoeconsumo.com.br/17/09/2022/econ...,17 de setembro de 2022,"A Natura & Co. sofreu, ontem, uma forte queda,..."
1490,Consumidor pós-pandemia espera experiências al...,https://mercadoeconsumo.com.br/17/09/2022/econ...,17 de setembro de 2022,Após as transformações causadas pela pandemia ...


## Export

In [22]:
# !pip install pyarrow

# df.to_csv('./data/df.txt', sep=';', index=False)
# df.to_feather('./data/df.feather') # [1]
df.to_parquet('./data/df.parquet', engine='pyarrow', compression='gzip', index=False) # [2]

# [1] usar caso precise de leitura e armazenamento rápido
# [2] para fins de armazenamento, como não será atualizado com frequência, melhor usar parquet

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html